# Quantum Circuits
**Note**: In this notebook, all of the images are taken from the website: https://pennylane.ai/codebook/introduction-to-quantum-computing/quantum-circuits

These notes introduce the quantum circuits, including:

- Visualizing quantum algorithms
- Wires and registers
- Quantum gates and operations
- Circuit depth
- Devices and QNodes

We will also implement examples using Python and PennyLane library.

## Visualizing quantum algorithms

Understanding qubits and their mathematical representations is one of the most important steps to discuss how quantum computations are expressed. Through the quantum computations, we can manipulate and measure these qubits in a meaningful way. However, we need to find a way to express quantum algorithms and protocols in order to understand these relations and representations without any particular hardware or programming language. We achieve this by **quantum circuits**.

Quantum circuits are a way to visually describe the sequence of operations on the qubits through computation. We can think of quantum circuits as a set of instructions that tell what and when to do to the qubits. Performing particular operations in a certain way, we can run different algorithms. 

![title](Images/quantum-circuits-1.png)

## Wires and registers

Initially, a circuit consists of a collection of **wires**, which are lines representing a set of qubits. Qubits are placed from up to bottom in the numerical order, traditionally. A group of qubits together is called a **quantum register**. 

![title](Images/quantum-circuits-wires.png)

The qubits must be initialized to some state in the beginning of the computation. If the state is not determined initially, a typical choice is to start all the qubits in the $\ket{0}$. 

![title](Images/quantum-circuits-register.png)

## Quantum Gates

**Gates** in a quantum circuits are the operations on qubits. Some gates operate on single qubit, while some affect multiple qubits. Quantum circuits are read from left to right.

Below diagram indicates three wires, a quantum register consisting of three qubits, and different shapes which denote different quantum gates. Firstly, we apply the triangle gate on the qubits $0$ and $2$. The rectangle gate act on both qubit $0$ and $1$. A circle acts on qubit $2$ followed by a rectangle gate acting on both qubit $1$ and $2$. Lastly, one can see that a pentagon and a circle acts on the qubits $0$ and $1$, respectively. 

![title](Images/quantum-circuits-gates-1.png)

Quantum operations acting on separate qubits can be applied in parallel. Considering the above diagram, the pentagon gate can be moved to the left at the same time the rectangle acts on the qubits $1$ and $2$. 

![title](Images/quantum-circuits-gates-2.png)

However, we can realize this process when there is an empty place in the circuit. For instance, we could not move the rectangle in the second layer to the left because the triangle must be placed first, even though the qubit $1$ does not do anything at this time (there is an empty place for the qubit $1$ in this layer). Therefore, we visualize the circuits as a sequence of time layers: 

![title](Images/quantum-circuits-layers.png)

#### Code example: order of operations

Suppose that we have a quantum circuit consisting of Hadamard, CNOT, and rotation gates $R_{x}$ and $R_{y}$. Remind that the gates of $R_{x}$ and $R_{y}$ are defined as rotation matrix: 

$$ R_{x}(\theta)= \begin{pmatrix} cos(\frac{\theta}{2}) && -isin(\frac{\theta}{2}) \\ -sin(\frac{\theta}{2}) && cos(\frac{\theta}{2}) \end{pmatrix}$$
$$ R_{y}(\theta)= \begin{pmatrix} cos(\frac{\theta}{2}) &&  -sin(\frac{\theta}{2}) \\ sin(\frac{\theta}{2}) && cos(\frac{\theta}{2}) \end{pmatrix}$$

CNOT gate is represented as $4 \times 4$ matrix acting on $2$-qubits: 
$$ CNOT = \begin{pmatrix} 1 && 0 && 0 && 0  \\ 0 && 1 && 0 && 0 \\ 0 && 0 && 0 && 1 \\ 0 && 0 && 1 && 0 \end{pmatrix}$$

We know that Hadamard gate is a $2 \times 2$ matrix acting on a $1$-qubit defined as follows:
$$ H= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 && 1 \\ 1 && -1 \end{pmatrix}$$

Let's illustrate our circuit: 

![title](Images/quantum-circuits-exc-1.png)

One can easily see that the $R_{x}$, $R_{y}$, and Hadamard gate (which is denoted as H) acts on single qubit while CNOT gate operates on two qubits. We construct the quantum circuits regarding the sequential layers. More precisely, firstly we set the first gate in the first layer. Then, in the same layer, we place other gates from up to bottom if they exist. After that, we continue other layers respectively.

Now, we implement a *my_circuit* function taking two parameters, which are the angles of rotation gates. The argument of *wires* indicates the list of qubits we operate gates on.

In [55]:
import pennylane as qml

def my_circuit(theta, phi):
    
    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)

    # the measurement; we return the probabilities of all possible output states
    return qml.probs(wires=[0, 1, 2])

As you have seen, we first apply the CNOT as a first gate in the first layer. In the next line, we implement the gate $R_{x}$, which is placed as second in the first layer. Then, we continue layer by layer and follow the gates from up to bottom in the same layer. 

### Circuit depth

Complexity and algorithms are very crucial for efficient code implementations in classical computers. Similarly, in quantum computers, the number and the type of gates are important metrics. Another crucial metric is **circuit depth**, which is defined as the number of time steps to run a quantum circuit. More precisely, circuit depth is the number of layers in a quantum circuit. For example, in our previous examples, the circuit depth was $4$. 

Considering the circuit gates as lego bricks, the length of the structure built gives the circuit depth. For instance, the below colourful bricks representing quantum gates show that the length of the structure is $6$, which means we build a quantum circuit whose depth is $6$.

![title](Images/quantum-circuits-depth.png)

**Example**: Let us draw a circuit diagram for $4$-qubit circuit (the diagram will have $4$ lines) regarding these instructions: 

- initialize all the qubits in $\ket{0}$
- a circle gate acting on the qubit $0$
- a circle gate acting on the qubit $2$
- a triangle gate acting on the qubit $2$
- a triangle gate acting on the qubit $3$
- a rectangle gate acting on both the $0$ and $1$
- a rectangle gate acting on both the $1$ and $2$
- a rectangle gate acting on both the $2$ and $3$
- measurement of all the qubits

Through these rules, we have a circuit diagram as follows: 

![title](Images/quantum-circuits-example-circuit.png)

As easily seen from the diagram, the circuit depth is $4$. Note that we count the layers, not the qubits (wires).

## Devices and QNodes

We constructed circuits, but did not run yet. A quantum function is not alone enough to run and execute a circuit. We will need 

- a device to run the circuit
- QNode, which binds the circuit to the device and execute it

We can construct a device using the PennyLane (we import it as *qml*) function `qml.device` with the arguments of device name and the wire (qubits) list:

```dev = qml.device('device.name', wires=num_qubits)```

Alternatively, we can state wires as a list of qubits(wires) and device name as `default.qubit`: 

```dev = qml.device('default.qubit', wires=[0,1,2,...])```

After having a device, we can construct a *QNode*. It is the main unit of computation in PennyLane. 

![title](Images/quantum-circuits-QNode.png)


We construct QNode implementing the `qml.QNode` function:

`my_qnode = qml.QNode(my_circuit, my_device)`

Once we create the QNode, actually we call a function which uses the same prameter as the quantum function upon we have built. More precisely, QNode is a **decorator**, which is a function taking another function as an argument and returns a new function.

**Example**: Let consider the previous circuit diagram: 

![title](Images/quantum-circuits-exc-1.png)

We constructed the function `my_circuit`, but did not run and execute the circuit. Through using device and QNode, we will do run and execute it. 

In [106]:
import pennylane as qml

# we have three qubits 0,1,2; thus the wires will be the list [0,1,2]
dev = qml.device('default.qubit', wires = [0,1,2])

def my_circuit(theta, phi):
    
    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)

    # the measurement; we return the probabilities of all possible output states
    return qml.probs(wires=[0, 1, 2])

# we create a QNode, binding the function and device
my_qnode = qml.QNode(my_circuit, dev)

# we set up some values for the input parameters
theta, phi = 0.1, 0.2 

# we execute the QNode by calling my_qnode, which is created as a function
my_qnode(theta, phi)

array([4.93780134e-01, 1.23651067e-03, 4.97090753e-03, 1.24480103e-05,
       4.93780134e-01, 1.23651067e-03, 4.97090753e-03, 1.24480103e-05])

Since we run a circuit with three qubits, we observe $2^{3}=8$ probabilites as a result of measurement.

**Example**: Suppose we have another circuit diagram: 

![title](Images/quantum-circuits-exc-2.png)


Firstly, we will set the rotation gates in the first layer; $R_{x}(\theta)$, $R_{y}(\phi)$, $R_{z}(\omega)$, respectively. After this, we will place two CNOT gates in the second and third layer. The last CNOT will be in the final layer, but note that the control and target bits are placed at the wire $2$ and $0$, respectively.  

In [108]:
# we set a device with three wires, determining the number of wires and type of device
dev = qml.device("default.qubit", wires=3)

def my_circuit(theta, phi, omega):

    # based on our algorithms, our gates will be constructed as follows
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    # control is at 2, target is at 0
    qml.CNOT(wires=[2,0])

    # the measurement; we return the probabilities of all possible output states
    return qml.probs(wires=[0, 1, 2])


# we create a QNode, binding the function and device
my_qnode = qml.QNode(my_circuit, dev)

# we set up some values for the input parameters
theta, phi, omega = 0.1, 0.2, 0.3

# we execute the QNode by calling my_qnode
my_qnode(theta, phi, omega)

array([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
       2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03])

Remind that QNode acts as a decorator. That's why, we can use decorator in our example as well. We can apply a decorator to the `my_circuit` to construct a QNode, then run it using the provided input parameters.

In [111]:
# we set a device with three wires, determining the number of wires and type of device
dev = qml.device("default.qubit", wires=3)

# apply the decorator to the my-circuit function
@qml.qnode(dev)
def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])

    # the measurement; we return the probabilities of all possible output states
    return qml.probs(wires=[0, 1, 2])

# we set up some values for the input parameters
theta, phi, omega = 0.1, 0.2, 0.3

# run the QNode with provided input parameters
my_circuit(theta, phi, omega)

array([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
       2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03])

We can see that the decorator `@qml.qnode(dev)` constructed QNode and returned a new function `my_circuit` without defining it again. Then, we used this returned function with the three input parameters to run the circuit. 

## Conclusion

In this notebook, we introduced the quantum circuits:

- The structure of quantum circuits and their fundamental units and concepts such as wires, registers, and circuit depths.
- How do we apply quantum gates and operations on qubits in a quantum circuit diagram.
- The concepts of devices and QNode to understand how to run and execute a qauntum circuit.
- Implementing some examples through PennyLane library and its particular functions.

In the next notes, we will explore unitary matrices. 

## References

Introduction to Quantum Computing: Quantum Circuits. PennyLane Codebook. https://pennylane.ai/codebook/introduction-to-quantum-computing/quantum-circuits

De Wolf, R. (2019). Quantum computing: Lecture notes. arXiv preprint arXiv:1907.09415.

Walter, M., & Ozols, M. (2022). Lectures Notes on Quantum Information Theory. University of Amsterdam, Tech. Rep.

Aaronson, S. (2022, May). Introduction to quantum information science II lecture notes.

Christandl, M. (2012, January). Quantum information theory.
